In [52]:
#!pip3 install pygal
#!pip3 install pygal_maps_fr
import pygal_maps_fr
import pygal
import requests
import pandas as pd
import plotly.graph_objects as go
import plotly.express as px
import plotly
from plotly.subplots import make_subplots
from datetime import datetime
from tqdm import tqdm
import imageio

import locale
locale.setlocale(locale.LC_ALL, 'fr_FR')

colors = px.colors.qualitative.D3 + px.colors.qualitative.Dark24 + plotly.colors.DEFAULT_PLOTLY_COLORS + px.colors.qualitative.Plotly + px.colors.qualitative.Alphabet

In [74]:
def download_data():
    url_metadata = "https://www.data.gouv.fr/fr/organizations/sante-publique-france/datasets-resources.csv"
    url_geojson = "https://raw.githubusercontent.com/gregoiredavid/france-geojson/master/departements.geojson"
        
    metadata = requests.get(url_metadata)
    geojson = requests.get(url_geojson)
    
    with open('data/france/metadata.csv', 'wb') as f:
        f.write(metadata.content)
        
    with open('data/france/dep.geojson', 'wb') as f:
        f.write(geojson.content)
        
    df_metadata = pd.read_csv('data/france/metadata.csv', sep=";")
    url_data = df_metadata[df_metadata['url'].str.contains("/donnees-hospitalieres-covid19")]["url"].values[0]
    
    data = requests.get(url_data)
    with open('data/france/donnes-hospitalieres-covid19.csv', 'wb') as f:
        f.write(data.content)

download_data()

In [76]:
def import_data():
    df = pd.read_csv('data/france/donnes-hospitalieres-covid19.csv', sep=";")
    df_regions = pd.read_csv('data/france/departments_regions_france_2016.csv', sep=",")
    df_reg_pop = pd.read_csv('data/france/population_regions.csv', sep=",")
    df_dep_pop = pd.read_csv('data/france/dep-pop.csv', sep=";")
    
    df = df.merge(df_regions, left_on='dep', right_on='departmentCode')
    df = df.merge(df_reg_pop, left_on='regionName', right_on='regionName')
    df = df.merge(df_dep_pop, left_on='dep', right_on='dep')
    #df_geojson = pd.read_csv('data/france/dep.geojson)
    
    df['rea_pop'] = df['rea']/df['regionPopulation']*100000
    df['rad_pop'] = df['rad']/df['regionPopulation']*100000
    df['dc_pop'] = df['dc']/df['regionPopulation']*100000
    df['hosp_pop'] = df['hosp']/df['regionPopulation']*100000
    return df
df = import_data()
df = df[df["sexe"] == 0]
df_region = df.groupby(['regionName', 'jour']).sum().reset_index()
#df_region["dc_cumsum"] = df.groupby(by=['regionName','jour']).sum().groupby(level=[0]).cumsum().reset_index()["dc"]
dates = list(dict.fromkeys(list(df['jour'].values))) 
df

,dep,sexe,jour,hosp,rea,rad,dc,departmentCode,departmentName,regionCode,regionName,regionPopulation,departmentPopulation,rea_pop,rad_pop,dc_pop,hosp_pop
0,01,0,2020-03-18,2,0,1,0,01,Ain,84.0,Auvergne-Rhône-Alpes,4130719,649012,0.000000,0.024209,0.0,0.048418
3,01,0,2020-03-19,2,0,1,0,01,Ain,84.0,Auvergne-Rhône-Alpes,4130719,649012,0.000000,0.024209,0.0,0.048418
6,01,0,2020-03-20,2,0,1,0,01,Ain,84.0,Auvergne-Rhône-Alpes,4130719,649012,0.000000,0.024209,0.0,0.048418
9,01,0,2020-03-21,4,0,1,0,01,Ain,84.0,Auvergne-Rhône-Alpes,4130719,649012,0.000000,0.024209,0.0,0.096835
12,01,0,2020-03-22,8,1,2,0,01,Ain,84.0,Auvergne-Rhône-Alpes,4130719,649012,0.024209,0.048418,0.0,0.193671
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2685,974,0,2020-03-22,2,2,0,0,974,La Réunion,4.0,La Réunion,451366,860896,0.443099,0.000000,0.0,0.443099
2688,974,0,2020-03-23,2,2,0,0,974,La Réunion,4.0,La Réunion,451366,860896,0.443099,0.000000,0.0,0.443099
2691,974,0,2020-03-24,3,3,0,0,974,La Réunion,4.0,La Réunion,451366,860896,0.664649,0.000000,0.0,0.664649
2694,974,0,2020-03-25,3,3,0,0,974,La Réunion,4.0,La Réunion,451366,860896,0.664649,0.000000,0.0,0.664649


In [140]:
fig = px.line(x=df_region['jour'], y=df_region['rea']+df_region['hosp'], color=df_region["regionName"], color_discrete_sequence=colors)
fig.update_layout(
    title={
                'text': "Nombre de <b>patients hospitalisés</b>",
                'y':0.95,
                'x':0.5,
                'xanchor': 'center',
                'yanchor': 'top'},
                titlefont = dict(
                size=20)
                 )
fig.update_xaxes(title="Jour")
fig.update_yaxes(title="Nb. de patients hospitalisés")
fig.write_image("images/charts/france/hosp_line.png", scale=3, width=1100, height=700)
fig.show()

In [141]:
df_rhin = df[df["dep"].isin(["67", "68"])]
fig = px.line(x=df_rhin['jour'], y=df_rhin['rea'], color=df_rhin["dep"], labels={'color':'Département'}, color_discrete_sequence=colors)

fig.update_layout(
    title={
                'text': "Nombre de <b>patients en réa. ou soins intensifs</b> en Ht et Bas Rhin",
                'y':0.95,
                'x':0.5,
                'xanchor': 'center',
                'yanchor': 'top'},
                titlefont = dict(
                size=20)
                 )
fig.update_xaxes(title="Jour")
fig.update_yaxes(title="Nb. de patients en réa. ou soins intensifs")
fig.write_image("images/charts/france/rea_rhin.png", scale=3, width=1100, height=700)
fig.show()

In [8]:
Guillaumeimport json
with open('data/france/dep.geojson') as response:
    depa = json.load(response)

def map():
    #last_day = df['jour'].max()
    images = []
    
    df_map = df
    df_map["hosp_rea"] = df["hosp"] + df["rea"]
    df_map["hosp_rea_pop"] = (df_map["hosp_rea"]/df_map["departmentPopulation"])*100000
    max_val = df_map["hosp_rea_pop"].max()
    i=0
    with imageio.get_writer('images/charts/france/dep-map.gif', mode='I', duration=0.4) as writer:
        
        for date in tqdm(dates):
            
            df_map = pd.melt(df, id_vars=['jour','dep'], value_vars=['hosp_rea_pop'])
            #df_map = df_bar[df_bar['jour']==last_day]
            #df_map = df_bar.groupby(['regionName', 'variable']).sum().reset_index()
            df_map=df_map[df_map["jour"] == date]

            data={}
            for dep in df_map["dep"].values:
                data[dep] = df_map[df_map["dep"] == dep]["value"]



            fig = px.choropleth(df_map, geojson=depa, locations='dep', color='value',
                                       color_continuous_scale="Reds",
                                       range_color=(0, max_val),
                                       #color_continuous_midpoint=df_map["value"].mean(),
                                       featureidkey="properties.code",
                                        scope='europe',
                                       labels={'value':'hospit./100k hab.'}
                                      )
            date_title = datetime.strptime(dates[-1], '%Y-%m-%d').strftime('%d %B')
            fig.update_geos(fitbounds="locations", visible=False)
            fig.update_layout(
                margin={"r":0,"t":0,"l":0,"b":0},
                
                title={
                'text': "{}".format(date_title),
                'y':0.95,
                'x':0.5,
                'xanchor': 'center',
                'yanchor': 'top'},
                titlefont = dict(
                size=30),
                #projection_rotation=dict(lon=30, lat=30, roll=30)
            annotations = [dict(xref='paper',
                yref='paper',
                x=0, y=1.05,
                showarrow=False,
                #text =''.format(today, str(data['date'].values[-1])[:10]))]
            )]

            )
            fig.update_geos(
                #center=dict(lon=-30, lat=-30),
                projection_rotation=dict(lon=5, lat=35, roll=0),
                #lataxis_range=[-50,20], lonaxis_range=[0, 200]
            )
            fig.write_image("images/charts/france/dep-map-img/{}.png".format(date), scale=1, width=900, height=700)
            #fig.show()

            image = imageio.imread("images/charts/france/dep-map-img/{}.png".format(date))
            writer.append_data(image)
            i+=1
            if i==len(dates):
                for k in range(5):
                    writer.append_data(image)
        #images.append(imageio.imread("images/charts/france/dep-map_{}.png".format(date)))
    #imageio.mimsave('"images/charts/france/dep-map.gif"', images)

map()

100%|██████████| 9/9 [00:57<00:00,  6.34s/it]


In [142]:

fig = px.line(x=df['jour'], y=df['rea'], color=df["regionName"], labels={'color':'Région (des dép.)'}, color_discrete_sequence=colors)
fig.update_layout(
    title_text=''
)
fig.update_layout(
    title={
                'text': "Nb. de <b>patients en réanimation</b> par département",
                'y':0.95,
                'x':0.5,
                'xanchor': 'center',
                'yanchor': 'top'},
                titlefont = dict(
                size=20)
                 )
fig.update_xaxes(title="")
fig.update_yaxes(title="Nb. de patients en réa. ou soins intensifs")
fig.write_image("images/charts/france/hosp_dep.png", scale=3, width=1100, height=700)
fig.show()

In [143]:

fig = px.line(x=df_region['jour'], y=df_region['rea_pop']+df_region['hosp_pop'], labels={'color':'Région'}, color=df_region["regionName"], color_discrete_sequence=colors)
fig.update_layout(
    title={
                'text': "Nb. de <b>patients hospitalisés</b> <b>par habitant</b>",
                'y':0.95,
                'x':0.5,
                'xanchor': 'center',
                'yanchor': 'top'},
                titlefont = dict(
                size=20)
                 )
fig.update_xaxes(title="Jour")
fig.update_yaxes(title="Nb. de patients hospitalisés/100k hab. (de ch. région)")
fig.write_image("images/charts/france/hosp_hab.png", scale=3, width=1100, height=700)
fig.show()

In [144]:
fig = px.line(x=df_region['jour'], y=df_region['dc'], color=df_region["regionName"], labels={'color':'Région'}, color_discrete_sequence=colors)
fig.update_layout(
    title={
                'text': "Nombre de <b>décès cumulés</b> par région",
                'y':0.95,
                'x':0.5,
                'xanchor': 'center',
                'yanchor': 'top'},
                titlefont = dict(
                size=20)
                 )
fig.update_xaxes(title="Jour")
fig.update_yaxes(title="Nb. de décès cumulés")
fig.write_image("images/charts/france/dc_cum_line.png", scale=3, width=1100, height=700)
fig.show()

In [147]:
fig = px.line(x=df_region['jour'], y = df_region['dc']/df_region['regionPopulation']*100000, labels={'color':'Région'}, color=df_region["regionName"], color_discrete_sequence=colors)

fig.update_layout(
    title={
                'text': "Nombre de <b>décès cumulés</b> par <b>habitant</b>",
                'y':0.95,
                'x':0.5,
                'xanchor': 'center',
                'yanchor': 'top'},
                titlefont = dict(
                size=20)
                 )

fig.update_xaxes(title="Jour")
fig.update_yaxes(title="Nb. décès cumulés / 100k hab. de chaq. région")
fig.write_image("images/charts/france/dc_cum_hab_line.png", scale=3, width=1100, height=700)
fig.show()

In [114]:
fig = px.bar(x=df_region['jour'], y = df_region['dc'], color=df_region["regionName"], labels={'color':'Région'}, color_discrete_sequence=colors)

fig.update_layout(
    title={
                'text': "Nombre de <b>décès cumulés</b> par région",
                'y':0.95,
                'x':0.5,
                'xanchor': 'center',
                'yanchor': 'top'},
                titlefont = dict(
                size=20)
                 )
fig.update_xaxes(title="")
fig.update_yaxes(title="Nb. de décès cumulés")
fig.show()

fig.write_image("images/charts/france/dc_cum_region.png", scale=3, width=1100, height=700)

In [148]:

#df_region4 = df_region.groupby("regionName", "jour").sum().reset_index()
fig = go.Figure()
fig.add_trace(go.Bar(
    x = df_region[df_region["jour"] == dates[-1]]['regionName'],
    y = df_region[df_region["jour"] == dates[-1]]['dc'],
    name = datetime.strptime(dates[-1], '%Y-%m-%d').strftime('%d %B'),
    marker_color='indianred'
)).update_xaxes(categoryorder="total descending")
fig.add_trace(go.Bar(
    x = df_region[df_region["jour"] == dates[-2]]['regionName'],
    y = df_region[df_region["jour"] == dates[-2]]['dc'],
    name = datetime.strptime(dates[-2], '%Y-%m-%d').strftime('%d %B'),
    marker_color='indianred',
    opacity=0.6
))
fig.add_trace(go.Bar(
    x = df_region[df_region["jour"] == dates[-3]]['regionName'],
    y = df_region[df_region["jour"] == dates[-3]]['dc'],
    name = datetime.strptime(dates[-3], '%Y-%m-%d').strftime('%d %B'),
    marker_color='indianred',
    opacity=0.4
))


# Here we modify the tickangle of the xaxis, resulting in rotated labels.
fig.update_layout(
    barmode='group', xaxis_tickangle=-45,
    
    title={
                'text': "<b>Décès cumulés</b> par région",
                'y':0.95,
                'x':0.5,
                'xanchor': 'center',
                'yanchor': 'top'},
                titlefont = dict(
                size=20),
    xaxis_title="",
    yaxis_title="Nb. de décès cumulés",
                 )
fig.write_image("images/charts/france/dc_cum_region_comp.png", scale=3, width=1100, height=700)
fig.show()

"""with open('index.html', 'r') as f:
    data = f.read()

with open('index copy3.html', 'w') as f:
    #f.write(data.format(plotly.offline.plot(fig, include_plotlyjs=False, output_type='div')))
    f.write(data.format(fig.to_json(), fig.to_json()))
"""


"with open('index.html', 'r') as f:\n    data = f.read()\n\nwith open('index copy3.html', 'w') as f:\n    #f.write(data.format(plotly.offline.plot(fig, include_plotlyjs=False, output_type='div')))\n    f.write(data.format(fig.to_json(), fig.to_json()))\n"

In [136]:
df_region3 = df_region.groupby("regionName").sum().reset_index()
fig = px.bar(x=df_region3['regionName'], y = df_region3['dc'], color_discrete_sequence=colors).update_xaxes(categoryorder="total descending")
fig.update_layout(
    title_text='Nombre de décès cumulés'
)
fig.update_xaxes(title="")
fig.update_yaxes(title="Nb. de décès cumulés")
fig.write_image("images/charts/france/dc_cum.png", scale=3, width=1100, height=700)
#fig.show()

#####


In [149]:
df_region3 = df_region.groupby("regionName").sum().reset_index()
fig = px.bar(x=df_region3['regionName'], y = df_region3['dc_pop'], color_discrete_sequence=colors).update_xaxes(categoryorder="total descending")
fig.update_layout(
    title_text='Nombre de décès cumulés par habitant'
)
fig.update_xaxes(title="")
fig.update_yaxes(title="Nb. de décès cumulés / 100k habitants de chaq. rég.")
fig.write_image("images/charts/france/dc_cum_hab.png", scale=3, width=1100, height=700)
#fig.show()

In [150]:
fig = go.Figure()
df_region3 = df_region.groupby("regionName").sum().reset_index()
fig = make_subplots(specs=[[{"secondary_y": True}]])
fig.add_trace(go.Bar(
    x=df_region3['regionName'], 
    y = df_region3['dc'],
    name = "Nombre décès cumulés",
    width=0.3,
    marker_color='indianred'
),
             secondary_y = False).update_xaxes(categoryorder="total descending")

fig.add_trace(go.Bar(
    x=df_region3['regionName'], 
    y = df_region3['dc_pop'],
    name = "Nb. décès cum./100k hab.",
    marker_color='indianred',
    opacity=0.6,
    width=0.3,
    offset=0.15
    
),
             secondary_y = True)

fig.update_layout(
    barmode='group', 
    xaxis_tickangle=-45,
    
    title={
                'text': "Comparaison des <b>décès cumulés</b> et <b>décès cumulés par habitant</b>",
                'y':0.95,
                'x':0.5,
                'xanchor': 'center',
                'yanchor': 'top'},
                titlefont = dict(
                size=20),
    xaxis_title="",
                 )
fig.update_yaxes(title_text="Nb. décès cumulés", secondary_y=False)
fig.update_yaxes(title_text="Nb. décès cumulés/100k hab.", secondary_y=True)
fig.write_image("images/charts/france/dc_cum_hab_nonhab_comp.png", scale=3, width=1100, height=700)
fig.show()

In [138]:
df_bar = pd.melt(df, id_vars=['jour', 'regionName'], value_vars=['rea', 'hosp'])
df_bar = df_bar.groupby(['jour', 'variable']).sum().reset_index()
fig = px.bar(df_bar, x='jour', y='value', color="variable")
fig.update_layout(
    title={
            'text': "Nombre de <b>personnes hospitalisées</b> et en <b>réanimation</b>",
            'y':0.95,
            'x':0.5,
            'xanchor': 'center',
            'yanchor': 'top'},
    titlefont = dict(
            size=20),
    xaxis_title="",                
    )
fig.data[-1].name = 'Réanimation'
fig.data[-2].name = 'Hospitalisation'
fig.update_xaxes(title="Jour")
fig.update_yaxes(title="Nb. de patients actuellement hospitalisés")
fig.write_image("images/charts/france/hosp_rea_temps.png", scale=3, width=1100, height=700)
fig.show()

In [152]:
last_day = df['jour'].max()
df_bar = pd.melt(df, id_vars=['jour','regionName'], value_vars=['rea_pop', 'hosp_pop'])
df_bar = df_bar[df_bar['jour']==last_day]
df_bar = df_bar.groupby(['regionName', 'variable']).sum().reset_index()

fig = px.bar(df_bar, x='regionName', y='value', color="variable").update_xaxes(categoryorder="total descending")

fig.update_layout(
    title_text='Nb. de <b>patients hospitalisés</b> et en <b>réanimation</b> par <b>habitant</b> ({})'.format(dates[-1]),
    title={
    'y':0.95,
    'x':0.5,
    'xanchor': 'center',
    'yanchor': 'top'}
),
titlefont = dict(
            size=40),GuillaumeGuillaumeGuillaume
fig.data[-1].name = 'Réanimation'
fig.data[-2].name = 'Hospitalisation'
fig.update_xaxes(title="")
fig.update_yaxes(title="Nb. de patients / 100k hab. de chaq. région")
fig.write_image("images/charts/france/hosp_rea_hab.png", scale=3, width=1100, height=700)
fig.show()